YOLO V3
===

# 1.概述

YOLO算法创新性地提出了将输入图片进行N*N的栅格化（每个小单元叫grid cell），然后将图片中某个对象的位置的预测任务交与该对象中心位置所在的grid cell的bouding box。简单理解的话，可以认为这也是一种很粗糙的区域推荐（region proposal），在训练的时候，我们通过grid cell的方式告诉模型，图片中对象A应该是由中心落在特定grid cell 的某个范围内的某些像素组成，模型接收到这些信息后就在grid cell周围以一定大小范围去寻找所有满足对象A特征的像素，经过很多次带惩罚的尝试训练后，它就能找到这个准确的范围了（说明不是瞎找，如滑动窗口），当然这个方位不仅是指长宽的大小范围，也包括小幅度的中心位置坐标变化，但是不管怎么变，中心位置不能越过该grid cell的范围。这大大限制了模型在图片中瞎找时做的无用功。这样将位置检测和类别识别结合到一个CNN网络中预测，即只需要扫描一遍（you only look once）图片就能推理出图片中所有对象的位置信息和类别。举例如下图。

![Images](...%0AAA==)

不管是YOLOv1还是v2、v3，其主要的核心还是以上所述，只是在bounding box的拟合方式、骨干网络的设计、模型训练的稳定性、精度方面有所提升罢了

# 2.训练
YOLO算法几乎是输入原图就直接预测出每个grid cell“附近”是否有某个对象和具体的box位置，那最终这个想法数学化后便体现在loss函数上。在提出loss函数之前要先了解三个概念
- anchor box
- 置信度(confidence)
- 对象条件类别概率(conditional class probabilities)

在网络最后的输出中，对于每个grid cell对应bounding box的输出有三类参数
- 对象的box参数，一共是四个值，即box的中心点坐标（x,y）和box的宽和高（w,h）
- 置信度，这是个区间在\[0,1\]之间的值
- 一组条件类别概率，都是区间在\[0,1\]之间的值，代表概率。

## 2.1.Anchor box

### 2.1.1.Anchor box的由来
anchor box(也称为bounding box prior)其实就是从训练集的所有ground truth box中统计出来的在训练集中最经常出现的几个box形状和尺寸，统计的方法就是k-means算法。比如，在某个训练集中最常出现的box形状有扁长的、瘦高的和宽高比例差不多的正方形这三种形状。我们可以预先将这些统计上的先验经验加入到模型中，这样模型在学习的时候，瞎找的可能性就更小了些，当然就有助于模型快速收敛了。

以前面提到的训练数据集中的ground truth box最常出现的三个形状为例，当模型在训练的时候我们可以告诉它，你要在grid cell 1附近找出的对象的形状要么是扁长的、要么是瘦高的、要么是长高比例差不多的正方形，你就不要再瞎试其他的形状了。anchor box其实就是对预测的对象范围进行约束，并加入了尺寸先验经验，从而可以有效解决对象多尺度的问题

当我们只对图片中一个对象（且图片中只有一个对象）进行box回归时，我们只需要一个box回归器，但是当我们对图片中多个对象进行回归时（甚至一个类别会有多个对象），这时使用多个box回归器预测多个对象位置时就会发生冲突，因为每个预测器都可能不受约束地预测图片中任何一个对象的位置和类别。这时，我们就可以使用anchor来对每个回归器进行约束，只让每个回归器负责一块独立区域内的对象box回归。以YOLO算法举例，每个grid cell的位置其实也可以看做是anchor的位置（这不同于SSD或者Faster RCNN的anchor），如果最终的输出为13x13，也即有13x13个grid cell，每个grid cell有三个anchor的话，整个模型就有13x13x3个回归器，每个回归器只负责相应grid cell附近的对象预测。

### 2.1.2.量化
想办法找出分别代表这些形状的宽和高，有了宽和高，尺寸比例即形状不就有了。YOLO作者的办法是使用k-means算法在训练集中所有样本的ground truth box中聚类出具有代表性形状的宽和高，作者将这种方法称作维度聚类（dimension cluster）。细心的读者可能会提出这个问题：到底找出几个anchor box算是最佳的具有代表性的形状。YOLO作者方法是做实验，聚类出多个数量不同anchor box组，分别应用到模型中，最终找出最优的在模型的复杂度和高召回率(high recall)之间折中的那组anchor box。理论上来说聚类数量，也就是anchor的数量越多，效果是越好的，考虑到速度的原因，yolo的作者每个尺度下选择了3个anchor，但是有3个尺度，所以实际上是9个anchor

### 2.1.3.绑定
让一个grid cell输出多个bounding box，然后每个bounding box负责预测不同的形状。比如前面例子中的3个不同形状的anchor box，我们的一个grid cell会输出3个参数相同的bounding box，第一个bounding box负责预测的形状与anchor box 1类似的box，其他两个bounding box依次类推。求出每个grid cell中每个anchor box与ground truth box的IOU(交并比)，IOU最大的anchor box对应的bounding box就负责预测该ground truth，也就是对应的对象，

那么怎么告诉模型第一个bounding box负责预测的形状与anchor box 1类似，第二个bounding box负责预测的形状与anchor box 2类似？YOLO的做法是不让bounding box直接预测实际box的宽和高(w,h)，而是将预测的宽和高分别与anchor box的宽和高绑定，这样不管一开始bounding box输出的(w,h)是怎样的，经过转化后都是与anchor box的宽和高相关，这样经过很多次惩罚训练后，每个bounding box就知道自己该负责怎样形状的box预测了。

$$
\begin{eqnarray*}
b_w&=&\alpha_we^{t_w} \tag{1} \\
b_h&=&\alpha_he^{t_h} \tag{2}
\end{eqnarray*}
$$

其中，$\alpha_w$和$\alpha_h$为anchor box的宽和高，$t_w$和$t_h$为bounding box直接预测出的宽和高，$b_w$和$b_h$为转换后预测的实际宽和高，这也就是最终预测中输出的宽和高。这个公式这么麻烦，为什么不能用如下这样的公式

$$
\begin{eqnarray*}
b_w&=&\alpha_w \times t_w \\
b_h&=&\alpha_h \times t_h
\end{eqnarray*}
$$

上面的公式虽然计算起来比较麻烦，但是在误差函数求导后还带有和参数，而且也好求导.既然提到了最终预测的宽和高公式，那我们也就直接带出最终预测输出的box中心坐标的计算公式，我们前面提到过box中心坐标总是落在相应的grid cell中的，所以bounding box直接预测出的和也是相对grid cell来说的，要想转换成最终输出的绝对坐标，需要下面的转换公式：

$$
\begin{eqnarray*}
b_x&=&\sigma(t_x)+c_x \tag{1} \\
b_y&=&\sigma(t_y)+c_y \tag{2}
\end{eqnarray*}
$$

其中，$\sigma(t_{x})$为sigmoid函数，$c_{x}$和$c_{y}$分别为grid cell方格左上角点相对整张图片的坐标。作者使用这样的转换公式主要是因为在训练时如果没有将$t_{x}$和$t_{y}$压缩到(0,1)区间内的话，模型在训练前期很难收敛。最终可以得出实际输出的box参数公式如下，这个也是在推理时将输出转换为最终推理结果的公式：

$$
\begin{eqnarray*}
b_x&=&\sigma(t_x)+c_x \tag{1} \\
b_y&=&\sigma(t_y)+c_y \tag{2} \\
b_w&=&\alpha_we^{t_w} \tag{3} \\
b_h&=&\alpha_he^{t_h} \tag{4}
\end{eqnarray*}
$$

- $b_x,b_y$:实际最终预测的中心坐标
- $t_x,t_y$:经过网络预测后的中心坐标
- $c_x,c_y$:当前方格左上角点相对整张图片的x坐标
- $b_w,b_h$:实际最终预测的边框的宽和高
- $\alpha_w,\alpha_h$:对应的anchor box的宽和高
- $t_w,t_h$:经过网络预测后的边框的宽和高

关于box参数的转换还有一点值得一提，作者在训练中并不是将$t_x$、$t_y$、$t_w$和$t_h$转换为$b_x$、$b_y$、$b_w$和$b_h$后与ground truth box的对应参数求误差，而是使用上述公式的逆运算将ground truth box的参数转换为与$t_x$、$t_y$、$t_w$和$t_h$对应的$g_x$、$g_y$、$g_w$和$g_h$，然后再计算误差，计算中由于sigmoid函数的反函数难计算，所以并没有计算sigmoid的反函数，而是计算输出对应的sigmoid函数值。

$$
\begin{eqnarray*}
\sigma(\hat{t}_x)&=&g_x-c_x \tag{1} \\
\sigma(\hat{t}_y)&=&g_y-c_y \tag{2} \\
\hat{t}_w&=&log\frac{g_w}{\alpha_w} \tag{3} \\
\hat{t}_h&=&log\frac{g_h}{\alpha_h} \tag{4}
\end{eqnarray*}
$$

## 2.2.置信度(confidence)

置信度是每个bounding box输出的其中一个重要参数，它的作用定义有两重：
- 代表当前box是否有对象的概率，注意，是对象，不是某个类别的对象，也就是说它用来说明当前box内只是个背景（backgroud）还是有某个物体（对象）
- 表示当前的box有对象时，它自己预测的box与物体真实的box可能的的值，注意，这里所说的物体真实的box实际是不存在的，这只是模型表达自己框出了物体的自信程度

不管哪重含义，都表示一种自信程度：框出的box内确实有物体的自信程度和框出的box将整个物体的所有特征都包括进来的自信程度。经过以上的解释，其实我们也就可以用数学形式表示置信度的定义了：

$$C_i^j=P_r(Object) \times IOU_{pred}^{truth}$$

其中，$C_i^j$表示第i个grid cell的第j个bounding box的置信度。

## 2.3.对象条件类别概率(conditional class probabilities)

对象条件类别概率是一组概率的数组，数组的长度为当前模型检测的类别种类数量，它的意义是当bounding box认为当前box中有对象时，要检测的所有类别中每种类别的概率，其实这个和分类模型最后使用softmax函数输出的一组类别概率是类似的，只是二者存在两点不同：

- YOLO的对象类别概率中没有background一项，也不需要，因为对background的预测已经交给置信度了，所以它的输出是有条件的，那就是在置信度表示当前box有对象的前提下，所以条件概率的数学形式为$P_r(class_i|Object)$;
- 分类模型中最后输出之前使用softmax求出每个类别的概率，也就是说各个类别之间是互斥的，而YOLOv3算法的每个类别概率是单独用逻辑回归函数(sigmoid函数)计算得出了，所以每个类别不必是互斥的，也就是说一个对象可以被预测出多个类别

假如一个图片被分割成S*S个grid cell，我们有B个anchor box，也就是说每个grid cell有B个bounding box, 每个bounding box内有4个位置参数，1个置信度，classes个类别概率，那么最终的输出维数是$S \times S \times B \times (4+1+classes)$

## 2.4.损失函数

$$
\begin{eqnarray*}
Loss&=&\sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{I}_{ij}^{obj}\{[\sigma(t_x)_i^j-\sigma(\hat{t}_x)_i^j]^2+[\sigma(t_y)_i^j-\sigma(\hat{t}_y)_i^j]^2\} \tag{1} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{I}_{ij}^{obj}\{[{t_w}_i^j-{\hat{t}_w}_i^j]^2+[{t_h}_i^j-{\hat{t}_h}_i^j]^2\} \tag{2} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{G}_{ij}(C_i^j-\hat{C}_i^j)^2 \tag{3} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\sum_{c \in classes}\mathbb{I}_{ij}^{obj}[p_i^j(c)-\hat{p}_i^j(c)]^2 \tag{4}
\end{eqnarray*}
$$

### 2.4.1.$\mathbb{I}_{ij}^{obj}$

在训练中，如果某个grid cell的bounding box没有负责预测某个对象，那我们就不应该训练该bounding box的条件类别概率和坐标参数，因为使用这些参数的前提是**明确清楚该bounding box负责预测某个ground truth box**，这个时候就需要参数了，当该bounding box负责预测某个ground truth box时，$\mathbb{I}_{ij}^{obj}=1$，否则，$\mathbb{I}_{ij}^{obj}=0$。

### 2.4.2.$\hat{C}_i^j$

训练中，$\hat{C}_i^j$的取值是由grid cell的bounding box有没有负责预测某个对象决定的。如果负责，那么$\hat{C}_i^j=1$，否则，$\hat{C}_i^j=0$。一个ground truth box可以对应多个anchor box，那么跟每一个anchor box都有一个IOU的值，那么如果某一个anchor box在所有的IOU中最大，那么这个anchor box就负责预测这个ground truth box，这时$\hat{C}_i^j=1$，其他情况下$\hat{C}_i^j=0$。

### 2.4.3.$\mathbb{G}_{ij}$

如果某一个anchor box不是对应的ground truth中负责预测的(也就是IOU不是最大的)，但是它的IOU却比设定的阈值还要大，这个时候忽略该anchor box所有输出的对loss的误差贡献，包括置信度误差，这是$\mathbb{G}_{ij}=0$，其它情况$\mathbb{G}_{ij}=1$。结合之前的说明可以看出，参数$\mathbb{I}_{ij}^{obj}$和$\widehat C_{i}^{j}$的值其实是保持一致的。 

### 2.4.4.$S$

自v2后，YOLO算法网络结构中只使用卷积和池化操作进行特征提取和推理运算，去掉了传统的全连接层，这样的做法有一个好处，就是理论上整个网络不再限制输入图片的尺寸，因为卷积层本来就对输入的尺寸没有限制。

在YOLOV3中，作者提出了Multi-Scale training，在输入数据时，每10个batch，就会更改一次图片输入的大小，那么具体的图片大小是从下面的尺寸中选择出来的$(10 \times 32, 11 \times 32, 12 \times 32, 13 \times 32, 14 \times 32, 15 \times 32, 16 \times 32)$。当确定了输入层的大小后，模型通过卷积层输入输出尺寸公式计算后，便能预先知道在某个输入尺寸前提下，最后的输出层尺寸是多少了，也就是grid cell的个数，即loss函数中S的值。其实，S的取值只可能在集合$(10,11,12,13,14,15,16)$中，因为YOLO模型的降采样(down-sample)因子是32。

## 2.5.跨尺寸预测(Predictions across scales)
YOLO算法从三个不同的尺寸预测对象box，这三个不同的尺寸来自不同层级的卷积层的输出。该方法借鉴了feature pyramid network的思想: 由于卷积层每隔几层，特征映射(feature mapping)的宽和高就会减少，而通道数会增加，随着网络层次的加深，特征映射组成的形状类似于金字塔，如果将不同层级的特征映射转换为最终的输出，那么将有助于提升模型在对象不同尺度大小上的表现，即有助于提高模型从小目标到大目标的综合检测（box的精度）能力，

![Images](...%0AAAA=)

- 从网络的不同层次映射不同尺寸的输出，如图中从79层（外加两个卷积层）得到13*13的的输出;从91层（外加两个卷积层）得到26*26的输出;最后再得到52*52的输出。
- 后面的高层结合使用低层特征（图中的86、98层，分别使用了61层和36层的特征映射），使高层能使用细粒度（fine grained）特征和更多的语义信息。
- 最后一个尺寸输出使用了前两个尺寸计算的特征映射，使得最后的尺寸输出也能使用细粒度。
- 每个YOLO输出层中，每个grid cell的bounding box数量为3，而不是9，这样不同的YOLO输出层便能负责不同尺寸大小的对象预测了，这个思想来自SSD。例如，COCO数据集中，作者让YOLO scale1负责预测的尺寸有(116,90)、(156,198)和(373,326), YOLO scale2负责预测的尺寸有(30,61)、(62,45)和(59,119)，YOLO scale3负责预测的尺寸有(10,13)、 (16,30)和 (33,23)

![Images](...%0AIAiIgCIiAIiIAiIgCIiAIiID/9k=)

![Images](...%0AAAAAAAAAAAAAAAAAAAAAAAAAAAA//9k=)

# 4.再谈边框回归
我们根据前面的介绍，可以知道，在YOLOV3的损失函数中，用于边框回归的四个参数如下

$$
\begin{eqnarray*}
\sigma(\hat{t}_x)&=&g_x-c_x \tag{1} \\
\sigma(\hat{t}_y)&=&g_y-c_y \tag{2} \\
\hat{t}_w&=&log\frac{g_w}{\alpha_w} \tag{3} \\
\hat{t}_h&=&log\frac{g_h}{\alpha_h} \tag{4}
\end{eqnarray*}
$$

同时，也知道损失函数如下

$$
\begin{eqnarray*}
Loss&=&\sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{I}_{ij}^{obj}\{[\sigma(t_x)_i^j-\sigma(\hat{t}_x)_i^j]^2+[\sigma(t_y)_i^j-\sigma(\hat{t}_y)_i^j]^2\} \tag{1} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{I}_{ij}^{obj}\{[{t_w}_i^j-{\hat{t}_w}_i^j]^2+[{t_h}_i^j-{\hat{t}_h}_i^j]^2\} \tag{2} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{G}_{ij}(C_i^j-\hat{C}_i^j)^2 \tag{3} \\
&+& \sum_{i=0}^{S^2}\sum_{j=0}^B\sum_{c \in classes}\mathbb{I}_{ij}^{obj}[p_i^j(c)-\hat{p}_i^j(c)]^2 \tag{4}
\end{eqnarray*}
$$

## 4.1.$\hat{t}_w$与$\hat{t}_h$

$\hat{t}_w$和$\hat{t}_h$的公式yolov3和faster-rcnn系列是一样的，是物体所在边框的长宽和anchor box长宽之间的比率，不管Faster-RCNN还是YOLO，都不是直接回归bounding box的长宽而是尺度缩放到**对数空间**，是怕训练会带来不稳定的梯度。因为如果不做变换，直接预测相对形变tw，那么要求$\hat{t}_w>0$，因为你的框的宽高不可能是负数。这样，是在做一个有不等式条件约束的优化问题，没法直接用SGD来做。所以先取一个对数变换，将其不等式约束去掉，就可以了。

## 4.2.计算误差的规则

负责预测ground truth的anchor box计算坐标误差、置信度误差(此时target为1)以及分类误差，而其它的anchor box只计算置信度误差(此时target为0)

## 4.3.微调
![Images](...%0AO/8TgirPmS6jgioAAAAASUVORK5CYII=)

有了平移$t_x,t_y$和尺度缩放$t_w,t_h$才能让anchor box经过微调与grand truth重合。如图，红色框为anchor box，绿色框为Ground Truth，平移+尺度缩放可实线红色框先平移到虚线红色框，然后再缩放到绿色框。边框回归最简单的想法就是通过平移加尺度缩放进行微调嘛。

边框回归为何只能微调？当输入的Proposal与Ground Truth相差较小时，即IOU很大时，可以认为这种变换是一种线性变换，那么我们就可以用线性回归（线性回归就是给定输入的特征向量 X, 学习一组参数 W, 使得经过线性回归后的值跟真实值 Y(Ground Truth)非常接近. 即Y≈WX ）来建模对窗口进行微调， 否则会导致训练的回归模型不work（当 Proposal跟 GT 离得较远，就是复杂的非线性问题了，此时用线性回归建模显然就不合理了）

Log函数明显不满足线性函数，但是为什么当Proposal与Ground Truth相差很小时，就可以认为是一种线性变化呢？我们在高数中学过如下极限:

$$\lim_{x=0}log(1+x)=x$$

那么就有

$$
\begin{eqnarray*}
\hat{t}_w&=&log\frac{g_x}{\alpha_w} \tag{1} \\
&=&log\frac{g_w+\alpha_w-\alpha_w}{\alpha_w} \tag{2} \\
&=&log(1+\frac{g_w-\alpha_w}{\alpha_w}) \tag{3}
\end{eqnarray*}
$$

当且仅当$g_w-\alpha_w \to 0$的时候，才会是线性函数

## 4.4.感受野

针对imageSize=416来说，最终的特征图为三个尺寸$13 \times 13, 26 \times 26, 52 \times 52$。
- 最小的$13 \times 13$特征图上，由于其感受野最大故用最大的anchor box$[116 \times 90, 156 \times 198, 373 \times326]$，适合检测较大的目标
- 中等的$26 \times 26$特征图上，由于其具有中等感受野故应用中等的anchor box$[30 \times 61，62 \times 45，59 \times 119]$，适合检测中等大小的目标
- 较大的$52 \times 52$特征图上，由于其具有较小的感受野故应用最小的anchor box$[10 \times 13，16 \times 30，33 \times 23]$，适合检测较小的目标